In [ ]:
%pip install --upgrade transformers accelerate sentencepiece optimum peft bitsandbytes

import torch
import torch.nn as nn
import torch.nn.functional as F

import transformers
from tqdm.auto import tqdm, trange

assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
model_name = "Enoch/llama-7b-hf"

# loading Llama tokenizer ...
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

# ... and the model itself
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    load_in_4bit=True,
    torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad = False

model.gradient_checkpointing_enable()  # only store a small subset of activations, re-compute the rest.
model.enable_input_require_grads()  # override an implementation quirk in gradient checkpoints that disables backprop unless inputs require grad
# more on gradient checkpointing: https://pytorch.org/docs/stable/checkpoint.html https://arxiv.org/abs/1604.06174

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/218 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


...

pytorch_model-00031-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00032-of-00033.bin:   0%|          | 0.00/405M [00:00<?, ?B/s]

pytorch_model-00033-of-00033.bin:   0%|          | 0.00/524M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

# Use PEFT

In [ ]:
num_prompts = 16

In [ ]:
import peft

assert isinstance(model.model.embed_tokens, nn.Embedding), "please reload the model"

peft_config = peft.PromptTuningConfig(
    task_type=peft.TaskType.CAUSAL_LM, num_virtual_tokens=num_prompts
)
model = peft.get_peft_model(
    model, peft_config
)  # note: for most peft methods, this line also modifies model in-place

model.enable_input_require_grads() # Enables the gradients for the input embeddings. This is useful for fine-tuning adapter weights while keeping the model weights fixed.
opt = torch.optim.AdamW(model.parameters(), lr=0.01)

print(
    "Trainable parameters:",
    sum(p.numel() for p in model.parameters() if p.requires_grad),
)
print(
    "Total parameters (excluding quantization):",
    sum(p.numel() for p in model.parameters()),
)

Trainable parameters: 65536
Total parameters (excluding quantization): 3500478464


In [ ]:
# prepare data
import datasets

num = 64
def reverse_phrases(phrase):
  reverse = ' '.join(phrase.split(' ')[::-1])
  return reverse

data = datasets.load_dataset("Abirate/english_quotes", split="train").shuffle()
data = data['quote'][:num]
data_truth = [phrase[1:-2]+ ' ' + reverse_phrases(phrase[1:-2]) for phrase in data]

In [ ]:
from tqdm.auto import tqdm as tqdma

In [ ]:
# train
epochs = 4
for epoch in range(epochs):
  for i in range(num):
    batch = tokenizer(data_truth[i], return_tensors="pt", return_token_type_ids=False).to(device)
    print(batch["input_ids"].shape)

    for j in trange(5):
      outputs = model(**batch)
      print(outputs.logits.shape)
      k = (batch["input_ids"].shape[1])//2
      next_word_logits = outputs.logits[:, num_prompts+k:-1, :]
      true_next_tokens = batch["input_ids"][:, k+1: ]
      loss = F.cross_entropy(
          next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1)
      )
      print("Loss:", loss)
      loss.backward()

      opt.step()
      opt.zero_grad()

      if loss.item() <= 0.1:
          break

      torch.cuda.empty_cache()

torch.Size([1, 87])


  0%|          | 0/5 [00:00<?, ?it/s]

torch.Size([1, 103, 32000])
Loss: tensor(3.4906, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 103, 32000])
Loss: tensor(3.4405, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 103, 32000])
Loss: tensor(3.3707, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 103, 32000])
Loss: tensor(3.3185, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 103, 32000])
Loss: tensor(3.2360, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 35])


  0%|          | 0/5 [00:00<?, ?it/s]

torch.Size([1, 51, 32000])
Loss: tensor(4.5334, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 51, 32000])
Loss: tensor(4.3641, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 51, 32000])
Loss: tensor(4.1917, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 51, 32000])
Loss: tensor(3.9612, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 51, 32000])
Loss: tensor(3.7577, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 57])


...

torch.Size([1, 71, 32000])
Loss: tensor(0.1085, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 71, 32000])
Loss: tensor(0.0782, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 65])


  0%|          | 0/5 [00:00<?, ?it/s]

torch.Size([1, 81, 32000])
Loss: tensor(0.3295, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 81, 32000])
Loss: tensor(0.2665, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 81, 32000])
Loss: tensor(0.1839, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 81, 32000])
Loss: tensor(0.1106, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 81, 32000])
Loss: tensor(0.0652, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 109])


  0%|          | 0/5 [00:00<?, ?it/s]

torch.Size([1, 125, 32000])
Loss: tensor(0.5666, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 125, 32000])
Loss: tensor(0.5026, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 125, 32000])
Loss: tensor(0.4332, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 125, 32000])
Loss: tensor(0.3783, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([1, 125, 32000])
Loss: tensor(0.3272, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
prompt = "This phrase should be inverse, I hope it will work"
batch = tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to(device)

for i in range(batch["input_ids"].shape[1]):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch["input_ids"] = torch.cat([batch["input_ids"], next_token], dim=-1)
    batch["attention_mask"] = torch.cat(
        [batch["attention_mask"], torch.ones_like(next_token)], dim=-1
    )

print(
    "\nOutput:",
    tokenizer.decode(batch["input_ids"][0, :].cpu().numpy().tolist()),
)


Output: <s>This phrase should be inverse, I hope it will work work will it hope I inverse, be should This phrase This
